In [46]:
import numpy as np
import pandas as pd
import requests
import os
import urllib.request
from bs4 import BeautifulSoup
import json
import openpyxl
import xlsxwriter

In [47]:
def AutoFitColumnSize(worksheet, columns=None, margin=2):
    for i, column_cells in enumerate(worksheet.columns):
        is_ok = False
        if columns == None:
            is_ok = True
        elif isinstance(columns, list) and i in columns:
            is_ok = True

        if is_ok:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length + margin

    return worksheet

In [48]:
page = 10
title = "koreapas.xlsx"

In [49]:
name_list = []
date_list = []
url_list = []
for num in range(1,page):
    url = "https://www.koreapas.com/bbs/zboard.php?id=circle&page=" + str(num) + "&select_arrange=headnum&desc=asc&category=&sn=off&ss=on&sc=on&tagkeyword=&keyword=&sn1=&divpage=3"
    html = urllib.request.urlopen(url).read()
    text = BeautifulSoup(html,"html.parser")
    table = text.find("table", id = "revolution_main_table")
    tr = table.find_all("tr")
    for i in range(len(tr)):
        if i%2 == 0:
            if i != 0:
                td = tr[i].find_all("td")
                name = td[2].text[3:]
                date = td[4].text
                k = td[2].find_all("a")
                name = name.replace("\ue251","")
                name = name.replace("\ue038","")
                name = name.replace("\xa0","")
                name_list.append(name)
                date_list.append(date)
                if len(k) == 1:
                    new_url = "https://www.koreapas.com/bbs/" + td[2].find("a")['href']
                else:
                    new_url = "https://www.koreapas.com/bbs/" + k[1]['href']
                url_list.append(new_url)
            else:
                continue

data = {"날짜" : date_list,"동아리 게시판" : name_list,  "url" : url_list}
df1 = pd.DataFrame(data)

In [50]:
name_list = []
date_list = []
url_list = []

for number in range(1,page):
    url = "https://www.koreapas.com/bbs/zboard.php?id=freead&page=" + str(number) +"&select_arrange=headnum&desc=asc&category=&sn=off&ss=on&sc=on&tagkeyword=&keyword=&sn1=&divpage=34"
    html = urllib.request.urlopen(url).read()
    text = BeautifulSoup(html,"html.parser")
    table = text.find("table", id = "revolution_main_table")
    tr = table.find_all("tr")
    for i in range(len(tr)):
        if i%2 == 0:
            if i == 2:
                continue
            elif i != 0:
                td = tr[i].find_all("td")
                name = td[2].text[3:]
                date = td[4].text
                name = name.replace("\ue251","")
                name = name.replace("\ue038","")
                name = name.replace("\xa0","")
                name_list.append(name)
                date_list.append(date)
                k = td[2].find_all("a")
                if len(k) == 1:
                    new_url = "https://www.koreapas.com/bbs/" + td[2].find("a")['href']
                else:
                    new_url = "https://www.koreapas.com/bbs/" + k[1]['href']
                url_list.append(new_url)
            else:
                continue
data = {"날짜" : date_list,"자유홍보" : name_list,  "url" : url_list}
df2 = pd.DataFrame(data)
df2[' '] = " "

In [51]:
df3 = pd.concat([df1,df2],axis=1)
df3[' '] = " "
df3.to_excel(title,engine = 'xlsxwriter')

In [52]:
wb = openpyxl.load_workbook(title)
sheet1 = wb.active
sheet1 = AutoFitColumnSize(sheet1,[1,2,4,5],2)
wb.save(title)
path = os.getcwd()
full_path = path + "/" + title
os.system("open -a '/Applications/Microsoft Excel.app' '%s'" %full_path)

0